In [ ]:
import sys

sys.path.append(r'C:\Users\sangeetha\Tract\Consulting\Neuropixel\NeuroPyxels')

In this I did the following

1. Edit folder_names in structure.oebin 
   1. *ProbeA to AP
   2. Acquisition* to LFP
2. Now reads meta data
3. Also gets good units


It runs from the code installed from cloned repo


What I should still test out

1. Spike times: I need spike_times.npy to test if spike times for any given unit can be loaded

In [ ]:
# %pip install seaborn imblearn h5py networkx cmcrameri scikit-learn matplotlib pandas tqdm cachecache scipy numba

In [ ]:
import npyx

In [ ]:
from npyx.inout import read_metadata

In [ ]:
dp = r'C:\Users\sangeetha\Data'
meta = read_metadata(dp)

In [ ]:
meta = {}

In [ ]:
meta

In [ ]:
import json

with open(r"C:\Users\sangeetha\Data\structure.oebin") as f:
    meta_oe = json.load(f)

In [ ]:
meta_oe

In [ ]:
for i,processor in enumerate(meta_oe['continuous']):
    if 'Neuropix-PXI' in processor["source_processor_name"]:
        probe_index = i
        break

In [ ]:
probe_index

In [ ]:
oe_probe_version = meta_oe["continuous"][probe_index]["source_processor_name"]

In [ ]:
meta_oe["continuous"][probe_index]["channels"][0]["bit_volts"]

In [ ]:
from npyx.utils import read_pyfile
from pathlib import Path

params_f = Path(r"C:\Users\sangeetha\Data\params.py")
if params_f.exists():
    params=read_pyfile(params_f)
params

In [ ]:
import os
from npyx.utils import list_files

filt_index = {'AB': [], 'NP': []}
for i,processor in enumerate(meta_oe['continuous']):
    if 'Acquisition_Board-104.Rhythm Data' in processor['folder_name']:
        filt_index['AB'] = i
    if 'Neuropix-PXI-100.ProbeA' in processor['folder_name']:
        filt_index['NP'] = i


# find everything else
for filt_key in ['AB','NP']:
    meta[filt_key]={}
    filt_key_i=filt_index[filt_key]
    meta[filt_key]['sampling_rate']=float(meta_oe["continuous"][filt_key_i]['sample_rate'])
    meta[filt_key]['n_channels_binaryfile']=int(meta_oe["continuous"][filt_key_i]['num_channels'])
    if params_f.exists():
        meta[filt_key]['n_channels_analysed']=params['n_channels_dat']
        meta[filt_key]['datatype']=params['dtype']
    else:
        meta[filt_key]['n_channels_analysed']=meta[filt_key]['n_channels_binaryfile']
        meta[filt_key]['datatype']='int16'
    binary_folder = './continuous/'+meta_oe["continuous"][filt_key_i]['folder_name']
    bin_folder_name = os.path.join(dp, binary_folder)
    binary_file = list_files(bin_folder_name, "dat", False)
    if any(binary_file):
        binary_rel_path = binary_folder+binary_file[0]
        meta[filt_key]['binary_relative_path']=binary_rel_path
        bin_rel_path_concatenate = os.path.join(dp, binary_rel_path)
        meta[filt_key]['binary_byte_size']=os.path.getsize(bin_rel_path_concatenate)
        if filt_key=='AB' and params_f.exists() and params['dat_path']!=binary_rel_path:
            print((f'\033[34;1mWARNING edit dat_path in params.py '
            f'so that it matches relative location of high pass filtered binary file: {binary_rel_path}'))
    else:
        meta[filt_key]['binary_relative_path']='not_found'
        meta[filt_key]['binary_byte_size']='unknown'
        print(f"\033[91;1mWARNING {filt_key} binary file not found at {dp}\033[0m")
    meta[filt_key]={**meta[filt_key], **meta_oe["continuous"][filt_key_i]}
meta["events"]=meta_oe["events"]
meta["spikes"]=meta_oe["spikes"]


In [ ]:
from npyx.gl import get_units
units = get_units(dp, quality='good')

In [ ]:
units

In [ ]:
from npyx.spk_t import trn
u=99
t = trn(dp, u)